### In this notebook, we will import the Groove Midi Dataset and process it, and save it for easily being used in our pipelines


In [1]:
%reload_ext autoreload

%autoreload 2
%matplotlib inline

In [2]:
# THIS SUPPRESSES THE TF WARNINGS
import warnings
warnings.filterwarnings('ignore')

We have the dataset already available in tensorflow_datasets, so let's load it first


In [38]:
import tensorflow as tf
# tfds works in both Eager and Graph modes
tf.enable_eager_execution() #not needed in TF V2, as it is already the default

import tensorflow_datasets as tfds

import numpy as np
import pickle

import pandas as pd

import ipyparams

from shutil import copy2


 ### NOTE:
 you can convert midi to note_sequence, two ways:
 1. USE magenta.music.midi_to_note_sequence, BUT THIS will GIVE YOU A PICKLING ERROR
 2. install note_seq (pip install note_seq) then use note_seq.midi_to_note_sequence

In [5]:
#import magenta.music as mm 
import note_seq
from magenta.models.music_vae import data



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [6]:
import magenta

print (magenta.__version__)

1.1.7


#### Let's Import OS and MIDI tools

In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
from datetime import datetime

from visual_midi import Plotter

from pretty_midi import PrettyMIDI

from IPython.core.display import display, HTML


# Load midi_utils from storage folder
import sys
sys.path.append("utils")
!ls
import midi_utils as mu

Groove MIDI Basics.ipynb     midi2HVO.ipynb
GrooveMiditoTfExamples.ipynb midi_to_hvo_processing.ipynb
Untitled1.ipynb              misc
_24_12_2020__16:32.obj       processed_dataset
__init__.py                  reference
main.py                      source_dataset
midi2HVO-Updated-Copy1.ipynb temp.txt
midi2HVO-Updated-Ver3.ipynb  utils
midi2HVO-Updated.ipynb


In [8]:
def add_line_to_text(text, new_line):
    return text+"\n"+new_line

In [9]:
info_text = ""
info_text = add_line_to_text(info_text, "THE EXISTING DATASET WAS PROCESSED TO NOTESEQUENCE ")
info_text = add_line_to_text(info_text, "AND HVO FORMAT FROM GROOVEMIDI_2BAR_HUMANIZE SET ")
info_text = add_line_to_text(info_text, "-"*50)
info_text = add_line_to_text(info_text, "-"*50)

print (info_text)



THE EXISTING DATASET WAS PROCESSED TO NOTESEQUENCE 
AND HVO FORMAT FROM GROOVEMIDI_2BAR_HUMANIZE SET 
--------------------------------------------------
--------------------------------------------------


In [10]:
#Source (code directly copy pasted from): note_seq/drums_encoder_decoder

# Default list of 9 drum types, where each type is represented by a list of
# MIDI pitches for drum sounds belonging to that type. This default list
# attempts to map all GM1 and GM2 drums onto a much smaller standard drum kit
# based on drum sound and function.
DEFAULT_DRUM_TYPE_PITCHES = [
    # kick drum
    [36, 35],

    # snare drum
    [38, 27, 28, 31, 32, 33, 34, 37, 39, 40, 56, 65, 66, 75, 85],

    # closed hi-hat
    [42, 44, 54, 68, 69, 70, 71, 73, 78, 80, 22],

    # open hi-hat
    [46, 67, 72, 74, 79, 81, 26],

    # low tom
    [45, 29, 41, 43, 61, 64, 84],

    # mid tom
    [48, 47, 60, 63, 77, 86, 87],

    # high tom
    [50, 30, 62, 76, 83],

    # crash cymbal
    [49, 52, 55, 57, 58],

    # ride cymbal
    [51, 53, 59, 82]
] 


#Source (code directly copy pasted from): magenta/models/music_vae/data.py

# 9 classes: kick, snare, closed_hh, open_hh, low_tom, mid_tom, hi_tom, crash,
# ride
REDUCED_DRUM_PITCH_CLASSES = DEFAULT_DRUM_TYPE_PITCHES
# 61 classes: full General MIDI set
FULL_DRUM_PITCH_CLASSES = [
    [p] for p in  # pylint:disable=g-complex-comprehension
    [36, 35, 38, 27, 28, 31, 32, 33, 34, 37, 39, 40, 56, 65, 66, 75, 85, 42, 44,
     54, 68, 69, 70, 71, 73, 78, 80, 46, 67, 72, 74, 79, 81, 45, 29, 41, 61, 64,
     84, 48, 47, 60, 63, 77, 86, 87, 50, 30, 43, 62, 76, 83, 49, 55, 57, 58, 51,
     52, 53, 59, 82]
]
ROLAND_DRUM_PITCH_CLASSES = [
    # kick drum
    [36],
    # snare drum
    [38, 37, 40],
    # closed hi-hat
    [42, 22, 44],
    # open hi-hat
    [46, 26],
    # low tom
    [43, 58],
    # mid tom
    [47, 45],
    # high tom
    [50, 48],
    # crash cymbal
    [49, 52, 55, 57],
    # ride cymbal
    [51, 53, 59]
]

instruments = ["kick","snare","hat_closed","hat_open","tom_low","tom_mid","tom_high","cymbal_crash","cymbal_ride"]

In [11]:
# ADD INFO TO TEXT FILE

Mapping_text = """DEFAULT_DRUM_TYPE_PITCHES = [
    # kick drum
    [36, 35],

    # snare drum
    [38, 27, 28, 31, 32, 33, 34, 37, 39, 40, 56, 65, 66, 75, 85],

    # closed hi-hat
    [42, 44, 54, 68, 69, 70, 71, 73, 78, 80, 22],

    # open hi-hat
    [46, 67, 72, 74, 79, 81, 26],

    # low tom
    [45, 29, 41, 43, 61, 64, 84],

    # mid tom
    [48, 47, 60, 63, 77, 86, 87],

    # high tom
    [50, 30, 62, 76, 83],

    # crash cymbal
    [49, 52, 55, 57, 58],

    # ride cymbal
    [51, 53, 59, 82]
] 


#Source (code directly copy pasted from): magenta/models/music_vae/data.py

# 9 classes: kick, snare, closed_hh, open_hh, low_tom, mid_tom, hi_tom, crash,
# ride
REDUCED_DRUM_PITCH_CLASSES = DEFAULT_DRUM_TYPE_PITCHES
# 61 classes: full General MIDI set
FULL_DRUM_PITCH_CLASSES = [
    [p] for p in  # pylint:disable=g-complex-comprehension
    [36, 35, 38, 27, 28, 31, 32, 33, 34, 37, 39, 40, 56, 65, 66, 75, 85, 42, 44,
     54, 68, 69, 70, 71, 73, 78, 80, 46, 67, 72, 74, 79, 81, 45, 29, 41, 61, 64,
     84, 48, 47, 60, 63, 77, 86, 87, 50, 30, 43, 62, 76, 83, 49, 55, 57, 58, 51,
     52, 53, 59, 82]
]
ROLAND_DRUM_PITCH_CLASSES = [
    # kick drum
    [36],
    # snare drum
    [38, 37, 40],
    # closed hi-hat
    [42, 22, 44],
    # open hi-hat
    [46, 26],
    # low tom
    [43, 58],
    # mid tom
    [47, 45],
    # high tom
    [50, 48],
    # crash cymbal
    [49, 52, 55, 57],
    # ride cymbal
    [51, 53, 59]
]

instruments = ["kick","snare","hat_closed","hat_open","tom_low","tom_mid","tom_high","cymbal_crash","cymbal_ride"]
"""

info_text = add_line_to_text(info_text, Mapping_text)
info_text = add_line_to_text(info_text, "-"*50)
info_text = add_line_to_text(info_text, "-"*50)

#### Let's Start With the 2bar-midionly dataset

In [12]:
dataset_train_unprocessed,dataset_train_info = tfds.load(
    name="groove/2bar-midionly",
    split=tfds.Split.TRAIN,
    try_gcs=True,
    with_info=True)

dataset_test_unprocessed = tfds.load(
    name="groove/2bar-midionly",
    split=tfds.Split.TEST,
    try_gcs=True)

dataset_validation_unprocessed = tfds.load(
    name="groove/2bar-midionly",
    split=tfds.Split.VALIDATION,
    try_gcs=True)



### NOTE: TF DATASETS ARE PYTHON ITERABLES
you can convert the dataset into a list or numpy array:

   * List(dataset)
   * tfds.as_numpy(dataset)
    
Lets see how many samples we've got in the dataset



In [13]:
 # Build your input pipeline

"""dataset_train = dataset_train.batch(4).prefetch(
    tf.data.experimental.AUTOTUNE)"""

dataset_train = dataset_train_unprocessed.batch(1)
dataset_test  = dataset_test_unprocessed.batch(1)
dataset_validation = dataset_validation_unprocessed.batch(1)

temp = list(dataset_train)
print("\n Number of Batches Created for Training  --> %d \n " % len(temp)) #convert iterable to list
del(temp)


 Number of Batches Created for Training  --> 18163 
 


#### Create a dict to store the processed data

* Keys from Groove MIDI Dataset:
    * bpm
    * drummer
    * id 
    * midi
    * style
    * time_signature
    * type
    
* Keys for processed data
    * note_sequence
    * hvo (Hits, Velocity, Offsets)
    * hv0 (Hits, Velocity, zero Offsets)
    * h0o 
    * h00
    * 0vo
    * 0v0
    * 00o

In [14]:
data_converter=data.GrooveConverter(
        split_bars=2, steps_per_quarter=4, quarters_per_bar=4,
        max_tensors_per_notesequence=20, humanize=True,
        pitch_classes=ROLAND_DRUM_PITCH_CLASSES,
        inference_pitch_classes=REDUCED_DRUM_PITCH_CLASSES)


In [15]:
print(len(list(dataset_train)))

18163


In [16]:
dataframe = pd.read_csv('source_dataset/groove/info.csv', delimiter = ',')
dataframe.head()

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,test
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,test
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,test
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543,test
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500,test


In [17]:
(dataframe[dataframe.id == "drummer1/eval_session/1"]["session"]).to_numpy()

array(['drummer1/eval_session'], dtype=object)

In [23]:
def dict_append(dictionary, key, vals):
    # Appends a value or a list of values to a key in a dictionary
    
    # if the values for a key are not a list, they are converted to a list and then extended with vals
    dictionary[key]=list(dictionary[key]) if not isinstance(dictionary[key], list) else dictionary[key]
    
    # if vals is a single value (not a list), it's converted to a list so as to be iterable
    vals = [vals] if not isinstance(vals, list) else vals
        
    # append new values 
    for val in vals:
        dictionary[key].append(val)

    return dictionary

def convert_groove_midi_dataset(dataset, csv_dataframe_info=None):
    
    dataset_dict_processed = dict()
    dataset_dict_processed.update({
        "drummer":[],
        "session":[],
        "loop_id":[],  # the id of the recording from which the loop is extracted
        "master_id":[], # the id of the recording from which the loop is extracted
        "style_primary":[],
        "style_secondary":[],
        "bpm":[],
        "beat_type":[],
        "time_signature":[],
        "full_midi_filename":[],
        "full_audio_filename":[],
        "midi":[],
        "note_sequence":[],
        "hvo":[],
    })

    count = 0 
    
    
    for features in dataset:
        
        #midi, ID, style_primary, style_secondary,  = features["midi"], features["id"], features["style"]["primary"], features["style"]["secondary"]

        # Features readily available in dataset
        
        
        # Features to be extracted from the dataset
        
        note_sequence = note_seq.midi_to_note_sequence(tfds.as_numpy(features["midi"][0]))
        hvo = data_converter.to_tensors(note_sequence).outputs
        if hvo: # if hvo is None, don't add it to the dictionary
            if not csv_dataframe_info.empty:

                # Master ID for the Loop
                main_id = features["id"].numpy()[0].decode("utf-8").split(":")[0]

                # Get the relevant series from the dataframe
                df = csv_dataframe_info[csv_dataframe_info.id == main_id]

            #dict_append(dataset_dict_processed, "bpm_master", df["bpm"].to_numpy()[0])


                dict_append(dataset_dict_processed, "drummer", df["drummer"].to_numpy()[0])
                dict_append(dataset_dict_processed, "session", df["session"].to_numpy()[0].split("/")[-1])
                dict_append(dataset_dict_processed, "loop_id", features["id"].numpy()[0].decode("utf-8"))
                dict_append(dataset_dict_processed, "master_id", main_id)

                style_full = df["style"].to_numpy()[0]
                style_primary = style_full.split("/")[0]
                dict_append(dataset_dict_processed, "style_primary", style_primary)

                if "/" in style_full:
                    style_secondary = style_full.split("/")[1]
                    dict_append(dataset_dict_processed, "style_secondary", style_secondary)
                else:
                    dict_append(dataset_dict_processed, "style_secondary", ["None"])

                dict_append(dataset_dict_processed, "bpm", df["bpm"].to_numpy()[0])
                dict_append(dataset_dict_processed, "beat_type", df["beat_type"].to_numpy()[0])
                dict_append(dataset_dict_processed, "time_signature", df["time_signature"].to_numpy()[0])
                dict_append(dataset_dict_processed, "full_midi_filename", df["midi_filename"].to_numpy()[0])
                dict_append(dataset_dict_processed, "full_audio_filename", df["audio_filename"].to_numpy()[0])
                #note_sequence = mm.midi_to_note_sequence(tfds.as_numpy(features["midi"][0]))
                dict_append(dataset_dict_processed, "midi", features["midi"].numpy()[0])
                dict_append(dataset_dict_processed, "note_sequence", [note_sequence])

            dict_append(dataset_dict_processed, "hvo", hvo[0])
            #h, v, o = np.hsplit(hvo[0],3)
            #dict_append(dataset_dict_processed, "h", h)
            #dict_append(dataset_dict_processed, "v", v)
            #dict_append(dataset_dict_processed, "o", o)
            

        else:
            pass 
            """dict_append(dataset_dict_processed, "hvo", [None]) 
            dict_append(dataset_dict_processed, "h", [None])
            dict_append(dataset_dict_processed, "v", [None])
            dict_append(dataset_dict_processed, "o", [None])"""
            
    return dataset_dict_processed

In [24]:
%%time

# Process Training Set
dataset_train = dataset_train_unprocessed.batch(1)
dataset_train_processed = convert_groove_midi_dataset(dataset_train, dataframe)

# Process Test Set
dataset_test = dataset_test_unprocessed.batch(1)
dataset_test_processed = convert_groove_midi_dataset(dataset_test, dataframe)

# Process Validation Set
dataset_validation = dataset_validation_unprocessed.batch(1)
dataset_validation_processed = convert_groove_midi_dataset(dataset_validation, dataframe)

CPU times: user 1min 47s, sys: 3.79 s, total: 1min 51s
Wall time: 1min 44s


In [25]:
%%time

info_text = add_line_to_text(info_text, "-"*50)
info_text = add_line_to_text(info_text, "-"*50)

# Let's do a preliminary check of the training set
for key in dataset_train_processed.keys():
    info_text = add_line_to_text(info_text, "Training Set\n\n\n")
    info_text = add_line_to_text(info_text, "Feature %s --> Size = %d" % (key, len(dataset_train_processed[key])))
    print("Feature %s --> Size = %d" % (key, len(dataset_train_processed[key])))

info_text = add_line_to_text(info_text, "-"*50)
info_text = add_line_to_text(info_text, "-"*50)

# Let's do a preliminary check of the test set
for key in dataset_test_processed.keys():
    info_text = add_line_to_text(info_text, "Test Set\n\n\n")
    info_text = add_line_to_text(info_text, "Feature %s --> Size = %d" % (key, len(dataset_test_processed[key])))
    print("Feature %s --> Size = %d" % (key, len(dataset_test_processed[key])))

info_text = add_line_to_text(info_text, "-"*50)
info_text = add_line_to_text(info_text, "-"*50)

# Let's do a preliminary check of the validation set
for key in dataset_validation_processed.keys():
    info_text = add_line_to_text(info_text, "Validation Set\n\n\n")
    info_text = add_line_to_text(info_text, "Feature %s --> Size = %d" % (key, len(dataset_validation_processed[key])))    
    print("Feature %s --> Size = %d" % (key, len(dataset_validation_processed[key])))

Feature drummer --> Size = 16389
Feature session --> Size = 16389
Feature loop_id --> Size = 16389
Feature master_id --> Size = 16389
Feature style_primary --> Size = 16389
Feature style_secondary --> Size = 16389
Feature bpm --> Size = 16389
Feature beat_type --> Size = 16389
Feature time_signature --> Size = 16389
Feature full_midi_filename --> Size = 16389
Feature full_audio_filename --> Size = 16389
Feature midi --> Size = 16389
Feature note_sequence --> Size = 16389
Feature hvo --> Size = 16389
Feature drummer --> Size = 2064
Feature session --> Size = 2064
Feature loop_id --> Size = 2064
Feature master_id --> Size = 2064
Feature style_primary --> Size = 2064
Feature style_secondary --> Size = 2064
Feature bpm --> Size = 2064
Feature beat_type --> Size = 2064
Feature time_signature --> Size = 2064
Feature full_midi_filename --> Size = 2064
Feature full_audio_filename --> Size = 2064
Feature midi --> Size = 2064
Feature note_sequence --> Size = 2064
Feature hvo --> Size = 2064
Feat

In [26]:
def sort_dictionary_by_key (dictionary_to_sort, key_used_to_sort):
    # sorts a dictionary according to the list within a given key
    sorted_ids=np.argsort(dictionary_to_sort[key_used_to_sort])
    for key in dictionary_to_sort.keys():
        dictionary_to_sort[key]=[dictionary_to_sort[key][i] for i in sorted_ids]
    return dictionary_to_sort

In [27]:
# Sort the sets using ids
dataset_train_processed = sort_dictionary_by_key(dataset_train_processed, "loop_id")
dataset_test_processed = sort_dictionary_by_key(dataset_test_processed, "loop_id")
dataset_validation_processed = sort_dictionary_by_key(dataset_validation_processed, "loop_id")


In [54]:
# DUMP INTO A PICKLE FILE
def store_dataset_as_pickle(dataset_list, 
                            filename_list, 
                            root_dir = "processed_dataset",
                            append_datetime=True, 
                            features_with_separate_picklefile = ["hvo", "midi", "note_seq"]
                           ):

    #filename = filename.split(".obj")[0]
    path = root_dir
    
    if append_datetime:
        now = datetime.now()
        dt_string = now.strftime("%d_%m_%Y_at_%H_%M_hrs")
    else:
        dt_string =""
        
    path = os.path.join(path, "Processed_On_"+dt_string)
    
    if not os.path.exists (path):
        os.makedirs(path)

    currentNotebook = ipyparams.notebook_name
    print("Copying Source Code from %s to %s" % (os.path.join(os.getcwd(), currentNotebook), path))
    print("-"*100)

    copy2(os.path.join(os.getcwd(), currentNotebook), path) 
    
    for i, dataset in enumerate(dataset_list):

        subdirectory = os.path.join(path, filename_list[i])
        if not os.path.exists (subdirectory):
            os.makedirs(subdirectory)
            
        print("-"*100)
        print("-"*100)
        print("Processing %s folder" % subdirectory)
        print("-"*100)
        print("-"*100)
        
        # Create Metadata File
        csv_dataframe = pd.DataFrame()
        
        for k in dataset.keys():
            if k not in features_with_separate_picklefile:
                csv_dataframe[k] = dataset[k]
        
        csv_dataframe.to_csv(os.path.join(subdirectory, "metadata.csv"))
        
        print("Metadata created!")
        print("-"*100)

        for feature in features_with_separate_picklefile:
            if feature in dataset.keys():
                dataset_filehandler = open(os.path.join(subdirectory, "%s_data.obj"%feature),"wb")
                pickle.dump(dataset[feature],  dataset_filehandler)
                dataset_filehandler.close()
                print("feature %s pickled at %s" % (feature, os.path.join(subdirectory, "%s.obj"%filename_list[i].split(".")[0])))
                print("-"*100)

            else:
                 raise Warning("Feature is not available: ", feature)


In [55]:
dataset_list = [dataset_train_processed,
               dataset_test_processed,
               dataset_validation_processed]

filename_list = ["GrooveMIDI_processed_train",
                "GrooveMIDI_processed_test",
                "GrooveMIDI_processed_validation"]

info_text = add_line_to_text(info_text, "-"*50)
info_text = add_line_to_text(info_text, "-"*50)

store_dataset_as_pickle(dataset_list, 
                        filename_list,
                        root_dir="processed_dataset",
                        append_datetime=True,
                        features_with_separate_picklefile = ["hvo", "midi", "note_sequence"]
                       )

Copying Source Code from /Users/behzadhaki/PycharmProjects/GrooveMIDIPreprocessing/midi2HVO-Updated-Ver3.ipynb to processed_dataset/Processed_On_17_01_2021_at_01_51_hrs
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Processing processed_dataset/Processed_On_17_01_2021_at_01_51_hrs/GrooveMIDI_processed_train folder
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Metadata created!
----------------------------------------------------------------------------------------------------
feature hvo pickled at processed_dataset/Processed_On_17_01_2021_at_01_51_hrs/GrooveMIDI_processed_train/Gr

In [77]:
source_path = "processed_dataset/Processed_On_17_01_2021_at_01_51_hrs"
print(os.path.join(source_path, "GrooveMIDI_processed_train", "hvo_data.obj"))
train_file = open(os.path.join(source_path, "GrooveMIDI_processed_train", "hvo_data.obj"),'rb')
train_set = pickle.load(train_file)
metadata = pd.read_csv(os.path.join(source_path, "GrooveMIDI_processed_train", "metadata.csv"))

features_in_metadata = list(metadata.columns)

dataset_size = len(train_set)

print("Dataset Size: %d \n Features: " % dataset_size, features_in_metadata)

processed_dataset/Processed_On_17_01_2021_at_01_51_hrs/GrooveMIDI_processed_train/hvo_data.obj
Dataset Size: 16389 
 Features:  ['Unnamed: 0', 'drummer', 'session', 'loop_id', 'master_id', 'style_primary', 'style_secondary', 'bpm', 'beat_type', 'time_signature', 'full_midi_filename', 'full_audio_filename']


In [78]:
ix =  int(np.random.random_sample()*dataset_size)
print("Sample Number: %d, Primary Style: %s, Secondary Style: %s" % (ix, 
                                                                     metadata["style_secondary"][ix], 
                                                                     metadata["style_primary"][ix])
     )


Sample Number: 8493, Primary Style: brazilian-samba, Secondary Style: latin
